Encoder Training
==============

In [ ]:
%pylab notebook

import tensorflow as tf
from tensorflow.keras import layers

import primo.models
import primo.datasets

import cupyck

Reserve space on the GPU for running simulations. It's important to do this before running any tensorflow code (which will take all available GPU memory):

In [ ]:
cupyck_sess = cupyck.GPUSession(max_seqlen=200, nblocks=1000, nthreads=128)

In [ ]:
simulator = primo.models.Simulator(cupyck_sess)

Load up the training and validation datasets:

In [ ]:
train_dataset = primo.datasets.OpenImagesTrain(
    '/tf/open_images/train/', switch_every=10**5
)

val_dataset = primo.datasets.OpenImagesVal('/tf/open_images/validation/')

In [ ]:
def keras_batch_generator(dataset_batch_generator, sim_thresh):
    while True:
        indices, pairs = next(dataset_batch_generator)
        distances = np.sqrt(np.square(pairs[:,0,:] - pairs[:,1,:]).sum(1))
        similar = (distances < sim_thresh).astype(int)
        yield pairs, similar

In [ ]:
sim_thresh = 75
encoder_train_batch_size = 100
encoder_val_batch_size = 2500
predictor_train_batch_size = 1000

encoder_train_batches = keras_batch_generator(
    train_dataset.balanced_pairs(encoder_train_batch_size, sim_thresh),
    sim_thresh
)

encoder_val_batches = keras_batch_generator(
    val_dataset.random_pairs(encoder_val_batch_size),
    sim_thresh
)

predictor_train_batches = train_dataset.random_pairs(1000)

Create the models and stack them together with the trainer:

In [ ]:
encoder = primo.models.Encoder()
yield_predictor = primo.models.Predictor('/tf/primo/data/models/yield-model.h5')
encoder_trainer = primo.models.EncoderTrainer(encoder, yield_predictor)

Run the training!

In [ ]:
encoder_trainer.model.compile(tf.keras.optimizers.Adagrad(1e-3), 'binary_crossentropy')

In [ ]:
history = encoder_trainer.model.fit_generator(
    encoder_train_batches,
    steps_per_epoch = 1000,
    epochs = 100,
    callbacks = [
        encoder_trainer.refit_predictor(
            predictor_train_batches, simulator, refit_every=1, refit_epochs=10
        )
    ],
    validation_data = encoder_val_batches,
    validation_steps = 1,
    verbose = 2
)

Save the models:

In [ ]:
encoder.save('/tf/primo/data/models/encoder_model.h5')
predictor.save('/tf/primo/data/models/predictor_model.h5')